# 零基础实战机器学习

## 第18讲 增长模型裂变实战

作者 黄佳

极客时间专栏链接：https://time.geekbang.org/column/intro/438


问题：判断哪种裂变带来更大的增长幅度

营销团队为易速鲜花的用户设计了两种裂变方案——“情侣花享”和“拼团盛放”，一个是类似于买一送一的促销，让用户把促销分享给自己的另一半。另一个裂变方案是用过生成专属海报进行拼团购买，团越大促销折扣越大。

那么，如何让机器自动的识别出，某一种裂变（或者说其它促销方案）所最易感的人群，然后把该裂变发给他或她呢？


## 数据导入

In [ ]:
import pandas as pd #导入Pandas
import numpy as np #导入NumPy
df_fission = pd.read_csv('易速鲜花增长模型.csv') #载入数据
print('用户数:', df_fission.count()['用户码']) #查看数据条目数
df_fission.head() #显示头几行数据

## 可视化

In [ ]:
import matplotlib.pyplot as plt #导入pyplot模块
import seaborn as sns #导入Seaborn
fig = sns.countplot('裂变类型', data=df_fission) #创建柱状计数图
fig.set_xticklabels(fig.get_xticklabels(),rotation=25) #X轴标签倾斜
fig.set_ylabel("数目") #Y轴标题
plt.show() #显示图像

In [ ]:
df_plot = df_fission.groupby('裂变类型').是否转化.mean().reset_index() #促销分组的转化率均值
plt.bar(df_plot['裂变类型'],df_plot['是否转化']) #不同促销转化均值柱状图

## 创建哑变量

In [ ]:
df_dummies = df_fission.drop(['裂变类型'],axis=1) #在拆分哑变量前，先拿掉裂变类型
df_dummies = pd.get_dummies(df_dummies, drop_first = True) #为分类变量拆分哑变量
df_dummies['裂变类型'] = df_fission['裂变类型'] #把裂变类型放回去
df_fission = df_dummies.copy() #把哑变量数据集复制给元数据集
df_fission.head() #显示数据

### 构建特征和标签数据集

In [ ]:
df_discount = df_fission.query("裂变类型 == '情侣花享' | 裂变类型 == '没有促销'")

In [ ]:
df_discount.loc[(df_discount.裂变类型 == '情侣花享') & (df_discount.是否转化 == 1), '标签'] = 0 #有应答裂变组,裂变购买者
df_discount.loc[(df_discount.裂变类型 == '情侣花享') & (df_discount.是否转化 == 0), '标签'] = 1 #无应答裂变组,裂变未购买者
df_discount.loc[(df_discount.裂变类型 == '没有促销') &  (df_discount.是否转化 == 1), '标签'] = 2 #有应答控制组,无裂变购买者
df_discount.loc[(df_discount.裂变类型 == '没有促销') &  (df_discount.是否转化 == 0), '标签'] = 3 #无应答控制组,无裂变未购买者
df_discount.head()

In [ ]:
X = df_discount.drop(['标签','是否转化'],axis=1) #特征集，Drop掉便签相关字段
y = df_discount.标签 #标签集

### 拆分数据集

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=16)

In [ ]:
X

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)

In [ ]:
import xgboost as xgb #导入xgboost模型
xgb_model = xgb.XGBClassifier() #创建xgboost模型
xgb_model.fit(X_train.drop(['用户码','裂变类型'], axis=1), y_train) #拟合xgboost模型

In [ ]:
uplift_probs = xgb_model.predict_proba(X_test.drop(['用户码','裂变类型'], axis=1)) #预测测试集用户的分类概率
uplift_probs #显示4种概率

## 求出增量分数

In [ ]:
discount_uplift = X_test.copy() #构建增量分数数据集   
discount_uplift['P_TR'] = uplift_probs[:,0] #添加有应答裂变概率
discount_uplift['P_TN'] = uplift_probs[:,1] #添加无应答裂变概率
discount_uplift['P_CR'] = uplift_probs[:,2] #添加有应答控制概率
discount_uplift['P_CN'] = uplift_probs[:,3] #添加无应答控制概率
#计算增量分数
discount_uplift['增量分数'] = discount_uplift.eval('P_TR + P_CN - (P_TN + P_CR)') 
# discount_uplift['增量分数'] = discount_uplift.eval('P_CN/(P_CN+P_CR) + P_TR/(P_TN+P_TR) - (P_TN/(P_TN+P_TR) - P_CR/(P_CN+P_CR))') 
discount_uplift